In [2]:
import pandas as pd
import tensorflow as tf

In [4]:
import matplotlib.pyplot as plt

In [7]:
zinc = pd.read_csv('../datasets/103_prop.csv')
zinc.head()

,ZINC_ID,MWT,LogP,Desolv_apolar,Desolv_polar,HBD,HBA,tPSA,Charge,NRB,SMILES
0,ZINC00000010,217.200,1.42,5.57,-41.98,0,4,66,-1,2,C[C@@]1(C(=O)C=C(O1)C(=O)[O-])c2ccccc2
1,ZINC00000012,289.356,1.28,4.89,-24.55,2,4,66,0,5,c1ccc(cc1)C(c2ccccc2)[S@](=O)CC(=O)NO
2,ZINC00000017,281.337,1.33,3.06,-23.33,2,6,87,0,4,CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N\C(=O)OC)/[nH]2
3,ZINC00000017,281.337,1.33,3.07,-19.20,2,6,87,0,4,CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N/C(=O)OC)/[nH]2
4,ZINC00000022,218.276,3.21,0.47,-48.57,1,3,52,-1,5,C[C@@H](c1ccc(cc1)NCC(=C)C)C(=O)[O-]


In [6]:
deg = pd.read_csv('../datasets/parsed_biodeg.csv')
deg.head()

,Unnamed: 0,SpMax_L: Leading eigenvalue from Laplace matrix,J_Dz(e): Balaban-like index from Barysz matrix weighted by Sanderson electronegativity,nHM: Number of heavy atoms,F01[N-N]: Frequency of N-N at topological distance 1,F04[C-N]: Frequency of C-N at topological distance 4,NssssC: Number of atoms of type ssssC,nCb-: Number of substituted benzene C(sp2),C%: Percentage of C atoms,nCp: Number of terminal primary C(sp3),...,C-026: R--CX--R,F02[C-N]: Frequency of C - N at topological distance 2,nHDon: Number of donor atoms for H-bonds (N and O),SpMax_B(m): Leading eigenvalue from Burden matrix weighted by mass,Psi_i_A: Intrinsic state pseudoconnectivity index - type S average,nN: Number of Nitrogen atoms,SM6_B(m): Spectral moment of order 6 from Burden matrix weighted by mass,nArCOOR: Number of esters (aromatic),nX: Number of halogen atoms,experimental class: ready biodegradable (RB) and not ready biodegradable (NRB)
0,0,4.170,2.1144,0,0,0,0,0,30.8,1,...,0,0,0,3.315,1.967,0,7.257,0,0,RB
1,1,3.932,3.2512,0,0,0,0,0,26.7,2,...,0,0,1,3.076,2.417,0,7.601,0,0,RB
2,2,3.000,2.7098,0,0,0,0,0,20.0,0,...,0,0,1,3.046,5.000,0,6.690,0,0,RB
3,3,4.236,3.3944,0,0,0,0,0,29.4,2,...,0,0,0,3.351,2.405,0,8.003,0,0,RB
4,4,4.236,3.4286,0,0,0,0,0,28.6,2,...,0,0,0,3.351,2.556,0,7.904,0,0,RB


In [9]:
deg.columns

Index(['Unnamed: 0', 'SpMax_L: Leading eigenvalue from Laplace matrix',
       'J_Dz(e): Balaban-like index from Barysz matrix weighted by Sanderson electronegativity',
       'nHM: Number of heavy atoms',
       'F01[N-N]: Frequency of N-N at topological distance 1',
       'F04[C-N]: Frequency of C-N at topological distance 4',
       'NssssC: Number of atoms of type ssssC',
       'nCb-: Number of substituted benzene C(sp2)',
       'C%: Percentage of C atoms', 'nCp: Number of terminal primary C(sp3)',
       'nO: Number of oxygen atoms',
       'F03[C-N]: Frequency of C-N at topological distance 3',
       'SdssC: Sum of dssC E-states',
       'HyWi_B(m): Hyper-Wiener-like index (log function) from Burden matrix weighted by mass',
       'LOC: Lopping centric index',
       'SM6_L: Spectral moment of order 6 from Laplace matrix',
       'F03[C-O]: Frequency of C - O at topological distance 3',
       'Me: Mean atomic Sanderson electronegativity (scaled on Carbon atom)',
       'Mi:

In [ ]:
def generator(z,reuse=None):
    with tf.variable_scope('gen', reuse=reuse):
        hidden1=tf.layers.dense(inputs=z, units=128, activation=tf.nn.leaky_relu)
        hidden2=tf.layers.dense(inputs=hidden1, units=128, activation=tf.nn.leaky_relu)
        output=tf.layers.dense(inputs=hidden2, units = 784, activation=tf.nn.tanh)
        
        return output

def discriminator(x, reuse=None):
    with tf.variable_scope('dis', reuse=reuse):
        hidden1=tf.layers.dense(inputs=x, units=128, activation=tf.nn.leaky_relu)
        hidden2=tf.layers.dense(inputs=hidden1, units=128, activation=tf.nn.leaky_relu)
        logits=tf.layers.dense(hidden2, units=1)
        output=tf.sigmoid(logits)
        
        return output, logits

In [ ]:
tf.reset_default_graph()

real_molecules=tf.placeholder(tf.float32, shape=[None,784])
z = tf.placeholder(tf.float32, shape=[None,100])

G = generator(z)
D_output_real,D_logits_real = discriminator(real_molecules)
D_output_fake,D_logits_fake = discriminator(G, reuse=True)

In [ ]:
def loss_func(logits_in, labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in, labels=labels_in))
                          
D_real_loss = loss_func(D_logits_real,tf.ones_like(D_logits_real)*0.9)
D_fake_loss = loss_func(D_logits_fake, tf.zeros_like(D_logits_real))
D_loss=D_real_loss + D_fake_loss
                          
G_loss = loss_func(D_logits_fake, tf.ones_like(D_logits_fake))

In [ ]:
lr=0.001

tvars=tf.trainable_variables()  #returns all variables created(the two variable scopes) and makes trainable true
d_vars=[var for var in tvars if 'dis' in var.name]
g_vars=[var for var in tvars if 'gen' in var.name]

D_trainer=tf.train.AdamOptimizer(lr).minimize(D_loss,var_list=d_vars)
G_trainer=tf.train.AdamOptimizer(lr).minimize(G_loss,var_list=g_vars)

batch_size=100
epochs=100
init=tf.global_variables_initializer()

In [ ]:
samples = []

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        num_batches=############################ batch size
        for i in range (num_batches):
            bath=mnist.train.next_batch(batch_size)
            batch